In [2]:
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.spatial.distance import correlation

In [3]:
data=pd.read_csv('data_collaborative.csv')
placeInfo=pd.read_csv('data_content.csv')

In [4]:
data=pd.merge(data,placeInfo,left_on='itemId',right_on="itemId")
userIds=data.userId
userIds2=data[['userId']]

data.loc[0:10,['userId']]
data=pd.DataFrame.sort_values(data,['userId','itemId'],ascending=[0,1])
data

,userId,itemId,rating,timestamp,category,distance,duration,nearby_places,title,url,p_rating,count
8,9,1,NaN,878089716,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158
17,9,2,3.0,890881335,Heritage,6 Kms,1-2 Hours,None,Nahargarh Fort,/rajasthan/jaipur/nahargarh-fort,4.6,652
26,9,3,3.0,889502324,Heritage,13 Kms,2-3 Hours,"Jaigarh Fort (1 km by walk), Srijagat Siromani...",Amer Fort / Amber Fort,/rajasthan/jaipur/amer-fort-amber-fort,5.0,783
35,9,4,3.0,879525876,Pilgrimage,6 Kms,1-2 Hours,None,Birla Mandir,/rajasthan/jaipur/birla-mandir,4.9,482
44,9,5,NaN,879485532,Heritage,6 Kms,30 Mins,None,Hawa Mahal,/rajasthan/jaipur/hawa-mahal,4.2,890
...,...,...,...,...,...,...,...,...,...,...,...,...
243,1,28,NaN,883748450,Museum,14 Kms,2 Hours,None,Anokhi Museum Of Hand Printing,/rajasthan/jaipur/anokhi-museum-of-hand-printing,1.7,560
252,1,29,4.0,875747401,Heritage,5 Kms,30 Mins,None,Amar Jawan Jyoti,/rajasthan/jaipur/amar-jawan-jyoti,5.0,348
261,1,30,NaN,876861185,"Heritage, Pilgrimage",14.5 Kms,2 Hours,None,Digamber Jain Mandir Sanghiji,/rajasthan/jaipur/digamber-jain-mandir-sanghiji,3.7,48
270,1,31,3.0,877052329,"Heritage, Pilgrimage",7 Kms,30 Mins,None,Govind Devji Temple,/rajasthan/jaipur/govind-devji-temple,1.5,654


In [8]:
def favoritePlace(activeUser,N):
    topPlace=pd.DataFrame.sort_values(
        data[data.userId==activeUser],['rating'],ascending=[0])[:N]
    print(data)
    return list(topPlace.title)

print(favoritePlace(1,5))

     userId  itemId  rating  timestamp               category  distance  \
8         9       1     NaN  878089716               Wildlife     6 Kms   
17        9       2     3.0  890881335               Heritage     6 Kms   
26        9       3     3.0  889502324               Heritage    13 Kms   
35        9       4     3.0  879525876             Pilgrimage     6 Kms   
44        9       5     NaN  879485532               Heritage     6 Kms   
..      ...     ...     ...        ...                    ...       ...   
243       1      28     NaN  883748450                 Museum    14 Kms   
252       1      29     4.0  875747401               Heritage     5 Kms   
261       1      30     NaN  876861185   Heritage, Pilgrimage  14.5 Kms   
270       1      31     3.0  877052329   Heritage, Pilgrimage     7 Kms   
279       1      32     NaN  882075827                 Museum     1 Kms   

      duration                                      nearby_places  \
8    1-2 Hours                

In [7]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['userId'], columns=['itemId'])
userItemRatingMatrix

itemId,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,1.0,NaN,1.0,NaN,NaN,5.0,3.0,NaN,...,NaN,2.0,4.0,NaN,5.0,NaN,4.0,NaN,3.0,NaN
2,NaN,4.0,NaN,4.0,1.0,NaN,1.0,4.0,NaN,4.0,...,5.0,NaN,NaN,5.0,5.0,NaN,3.0,NaN,4.0,3.0
3,4.0,5.0,NaN,5.0,NaN,4.0,NaN,3.0,1.0,3.0,...,5.0,3.0,NaN,4.0,NaN,4.0,4.0,NaN,2.0,NaN
4,4.0,NaN,4.0,NaN,NaN,5.0,4.0,3.0,NaN,NaN,...,NaN,4.0,4.0,NaN,4.0,5.0,NaN,2.0,5.0,NaN
5,3.0,3.0,NaN,3.0,4.0,NaN,NaN,3.0,5.0,5.0,...,4.0,3.0,3.0,NaN,3.0,4.0,4.0,NaN,4.0,4.0
6,NaN,NaN,5.0,3.0,NaN,2.0,3.0,NaN,4.0,4.0,...,3.0,2.0,NaN,4.0,3.0,NaN,4.0,5.0,NaN,5.0
7,3.0,NaN,4.0,NaN,3.0,3.0,2.0,NaN,3.0,4.0,...,4.0,4.0,3.0,NaN,NaN,2.0,5.0,NaN,4.0,4.0
8,NaN,3.0,NaN,1.0,4.0,5.0,NaN,5.0,3.0,NaN,...,2.0,4.0,NaN,NaN,4.0,5.0,NaN,3.0,3.0,NaN
9,NaN,3.0,3.0,3.0,NaN,5.0,5.0,NaN,2.0,4.0,...,NaN,NaN,4.0,1.0,1.0,NaN,3.0,NaN,4.0,NaN


In [9]:
def similarity(user1,user2):
    try:
        user1=np.array(user1)-np.nanmean(user1)
        user2=np.array(user2)-np.nanmean(user2)
        commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
        if len(commonItemIds)==0:
            return 0
        else:
            user1=np.array([user1[i] for i in commonItemIds])
            user2=np.array([user2[i] for i in commonItemIds])
            return correlation(user1,user2)
    except ZeroDivisionError:
        print("You can't divide by zero!")

In [10]:
def nearestNeighbourRatings(activeUser,K):
    try:
        similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,columns=['Similarity'])
        for i in userItemRatingMatrix.index:
            similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],userItemRatingMatrix.loc[i])
        similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,['Similarity'],ascending=[0])
        nearestNeighbours=similarityMatrix[:K]
        neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
        predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
        for i in userItemRatingMatrix.columns:
            predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
            for j in neighbourItemRatings.index:
                if userItemRatingMatrix.loc[j,i]>0:
                    predictedRating += (userItemRatingMatrix.loc[j,i]-np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']
                predictItemRating.loc[i,'Rating']=predictedRating
    except ZeroDivisionError:
        print("You can't divide by zero!")            
    return predictItemRating


In [12]:
def topNRecommendations(activeUser,N):
    try:
        predictItemRating=nearestNeighbourRatings(activeUser,10)
        placeAlreadyWatched=list(userItemRatingMatrix.loc[activeUser]
                              .loc[userItemRatingMatrix.loc[activeUser]>0].index)
        predictItemRating=predictItemRating.drop(placeAlreadyWatched)
        topRecommendations=pd.DataFrame.sort_values(predictItemRating,
                                                ['Rating'],ascending=[0])[:N]
        topRecommendationTitles=(placeInfo.loc[placeInfo.itemId.isin(topRecommendations.index)])
    except ZeroDivisionError:
        print("You can't divide by zero!")
    return list(topRecommendationTitles.title)

print(topNRecommendations(2,5))

['Jaipur Zoo', 'Amer Fort / Amber Fort', 'Maharani Ki Chhatri', 'City Palace / Sawai Man Singh II Museum', 'Albert Hall Museum']


C:\Users\areeb\anaconda3\lib\site-packages\scipy\spatial\distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [33]:
activeUser=input("Enter userid: ")
print("The user's favorite places are: ")
print(favoritePlace(activeUser,5))
# print("The recommended places for you are: ")
# print(topNRecommendations(activeUser,activeUser))

Enter userid: 3
The user's favorite places are: 
[]
